In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px

# Title and File Upload
st.title("Dynamic Data Visualization Dashboard")
st.write("Upload an Excel file to dynamically visualize your data.")

# Upload Excel file
uploaded_file = st.file_uploader("Upload your Excel file", type=["xlsx", "xls"])
if uploaded_file:
    try:
        # Load the data
        df = pd.read_excel(uploaded_file)
        st.write("### Data Preview")
        st.dataframe(df)

        # Visualization options
        st.write("### Visualization Options")
        chart_type = st.selectbox("Select Chart Type", ["Bar", "Line", "Scatter", "Pie"])
        x_axis = st.selectbox("Select X-axis", df.columns)
        y_axis = st.selectbox("Select Y-axis", df.columns if chart_type != "Pie" else ["None"])

        # Generate chart based on user selection
        st.write("### Generated Chart")
        if chart_type == "Bar":
            fig = px.bar(df, x=x_axis, y=y_axis, title="Bar Chart")
        elif chart_type == "Line":
            fig = px.line(df, x=x_axis, y=y_axis, title="Line Chart")
        elif chart_type == "Scatter":
            fig = px.scatter(df, x=x_axis, y=y_axis, title="Scatter Plot")
        elif chart_type == "Pie":
            fig = px.pie(df, names=x_axis, title="Pie Chart")

        st.plotly_chart(fig)

    except Exception as e:
        st.error(f"Error loading file: {e}")

else:
    st.write("Upload an Excel file to get started.")



In [4]:
import os
import pkg_resources
import ast

def extract_imports(file_path):
    """
    Extract all imported modules from the Python file.
    """
    with open(file_path, "r") as f:
        tree = ast.parse(f.read(), filename=file_path)
    
    imports = set()
    for node in ast.walk(tree):
        if isinstance(node, ast.Import):
            for alias in node.names:
                imports.add(alias.name.split(".")[0])  # Only top-level imports
        elif isinstance(node, ast.ImportFrom):
            if node.module:
                imports.add(node.module.split(".")[0])  # Only top-level imports
    return imports

def get_installed_version(package_name):
    """
    Get the installed version of a package.
    """
    try:
        version = pkg_resources.get_distribution(package_name).version
        return version
    except pkg_resources.DistributionNotFound:
        return None

def update_requirements_from_script(script_path, output_file="requirements.txt"):
    """
    Extract imports from the given script, find their versions,
    and write to requirements.txt.
    """
    imports = extract_imports(script_path)
    dependencies = []

    for package in imports:
        version = get_installed_version(package)
        if version:
            dependencies.append(f"{package}=={version}")
        else:
            print(f"Warning: Package '{package}' not found in the environment. Skipping.")

    # Write to requirements.txt
    with open(output_file, "w") as f:
        f.write("\n".join(sorted(dependencies)))
    
    print(f"Updated {output_file} with the following dependencies:")
    print("\n".join(dependencies))


# Example Usage
script_path = "app.py"  # Path to your app.py file
update_requirements_from_script(script_path)


Updated requirements.txt with the following dependencies:
pyomo==6.8.2
streamlit==1.40.0
